### Import libraries and dataset

In [1]:
import numpy
import tensorflow as tf
import random
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.python.keras.utils.np_utils import to_categorical

### Preprocess data, normalize images

In [2]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

print("shape of X_train: ", X_train.shape)
print("shape of X_test: ", X_test.shape)

shape of X_train:  (60000, 28, 28)
shape of X_test:  (10000, 28, 28)


### Create ANN model

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

def init():
    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation= 'relu'),
    tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    return model

modelInfo = init()
modelInfo.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [4]:
model = init()
r = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4976 - accuracy: 0.8243 - val_loss: 0.4144 - val_accuracy: 0.8561
Epoch 2/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3754 - accuracy: 0.8643 - val_loss: 0.4121 - val_accuracy: 0.8525
Epoch 3/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3373 - accuracy: 0.8765 - val_loss: 0.3673 - val_accuracy: 0.8668
Epoch 4/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3144 - accuracy: 0.8853 - val_loss: 0.3591 - val_accuracy: 0.8720
Epoch 5/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2971 - accuracy: 0.8907 - val_loss: 0.3490 - val_accuracy: 0.8770
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2800 - accuracy: 0.8982 - val_loss: 0.3507 - val_accuracy: 0.8721
Epoch 7/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2679 - accuracy: 0.9006 - val_loss: 0.3467 - val_accuracy:

In [5]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 0s 809us/step - loss: 0.3408 - accuracy: 0.8794


[0.3407944440841675, 0.8794000148773193]

### Train model for 1 epoch and return trained models with current loss

In [6]:
def train(models):
  
    losses = []
     
    for i in range(len(models)):
        history = models[i].fit(x=X_train,y=y_train, epochs=1, validation_data=(X_test, y_test))
        losses.append(round(history.history['loss'][-1], 4))
        
    return models, losses


In [7]:
no_of_generations = 10
no_of_individuals = 10
mutate_factor = 0.1
individuals = []

layers = [1, 2]

### Define mutate function to perform mutation of weights

In [8]:
def mutate(new_individual):
    for layer in layers:
        for bias in range(len(new_individual.layers[layer].get_weights()[1])):
            n = random.random()
            if n < mutate_factor:
                new_individual.layers[layer].get_weights(
                )[1][bias] *= random.uniform(-0.5, 0.5)

    for layer in layers:
        for weight in new_individual.layers[layer].get_weights()[0]:
            n = random.random()
            if n < mutate_factor:
                for j in range(len(weight)):
                    if random.random() < mutate_factor:
                        new_individual.layers[layer].get_weights(
                        )[0][j] *= random.uniform(-0.5, 0.5)

    return new_individual

### Define crossover function to form new generation with better weights

In [9]:
def crossover(individuals_param):
    new_individuals = [individuals_param[0], individuals_param[1]]

    for j in range(2, no_of_individuals):
        if j < (no_of_individuals - 2):
            if j == 2:
                parentA = random.choice(individuals_param[:3])
                parentB = random.choice(individuals_param[:3])
            else:
                parentA = random.choice(individuals_param[:])
                parentB = random.choice(individuals_param[:])

            for j in layers:
                temp = parentA.layers[j].get_weights()[1]
                parentA.layers[j].get_weights()[1] = parentB.layers[j].get_weights()[1]
                parentB.layers[j].get_weights()[1] = temp

            new_individual = random.choice([parentA, parentB])

        else:
            new_individual = random.choice(individuals_param[:])

        new_individuals.append(mutate(new_individual))
        # new_individuals.append(new_individual)

    return new_individuals

### Define evolve function to sort current population

In [10]:
def evolve(individuals_param, losses_param):
    sorted_y_idx_list = sorted(range(len(losses_param)), key=lambda x: losses_param[x])
    individuals_param = [individuals_param[x] for x in sorted_y_idx_list]

    # winners = individuals[:6]

    new_individuals = crossover(individuals_param)

    return new_individuals


### Train individuals and evolve

In [11]:
for i in range(no_of_individuals):
        individuals.append(init())

for generation in range(no_of_generations):
    individuals, losses = train(individuals)
    print(losses)

    individuals = evolve(individuals, losses)

1875/1875 [==============================] - 3s 1ms/step - loss: 0.5038 - accuracy: 0.8225 - val_loss: 0.4726 - val_accuracy: 0.8326
[0.4993, 0.4982, 0.4934, 0.4952, 0.5006, 0.5014, 0.503, 0.502, 0.5006, 0.5038]
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3773 - accuracy: 0.8637 - val_loss: 0.3892 - val_accuracy: 0.8629
[0.371, 0.3737, 0.3743, 0.3356, 0.3725, 0.3371, 0.3374, 0.3084, 0.3738, 0.3773]
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3117 - accuracy: 0.8860 - val_loss: 0.3499 - val_accuracy: 0.8708
[0.2926, 0.2761, 0.2665, 0.3345, 0.2552, 0.3372, 0.2445, 0.2389, 0.3115, 0.3117]
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1883 - accuracy: 0.9291 - val_loss: 0.3466 - val_accuracy: 0.8879
[0.2293, 0.2222, 0.2157, 0.209, 0.2954, 0.2017, 0.1974, 0.2789, 0.1928, 0.1883]
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2454 - accuracy: 0.9068 - val_loss: 0.3275 - val_accuracy: 0.8889
[0.1834, 0.1783

In [12]:
model = individuals[0]
model.evaluate(X_test, y_test)

313/313 [==============================] - 0s 697us/step - loss: 0.6711 - accuracy: 0.8875


[0.6711329221725464, 0.887499988079071]

In [13]:
# p_test = model.predict(X_test).argmax(axis=1)
# print(X_test.shape)
# i = numpy.random.choice(X_test.shape[0], replace=False)
# plt.imshow(X_test[i], cmap= 'gray')
# plt.title("True label: %s Predicted: %s" % (y_test[i], p_test[i]))